In [ ]:
try:
    DATALAKE_API_KEY = dbutils.secrets.get(scope="brewai-iaq-forecast", key="DATALAKE-API-KEY")
    API_KEY = dbutils.secrets.get(scope="brewai-iaq-forecast", key="API-KEY")
except:
    DATALAKE_API_KEY = None
    API_KEY = None

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta

In [ ]:
def get_request_header():
    return {"x-api-key": API_KEY}

In [ ]:
def get_device_ids(return_response=False):
    response = requests.get(
        url=f"https://jov3dcr05d.execute-api.ap-southeast-2.amazonaws.com/v1/sensordata/sensors",
        headers=get_request_header(),
    )
    if return_response:
        return response
    devices = [el[0] for el in response.json()['Records']]
    return devices

In [ ]:
def get_device_data(device):
    print("Handle", device)
    data = requests.get(
        url=f"https://jov3dcr05d.execute-api.ap-southeast-2.amazonaws.com/v1/sensordata?from={start_ts}&to={end_ts}&device={device}",
        headers={"x-api-key": API_KEY},
    ).json()
    rows = data["Records"]
    while data["ExecutionInfo"].get("NextToken") is not None:
        data = requests.get(
            url=f"https://jov3dcr05d.execute-api.ap-southeast-2.amazonaws.com/v1/sensordata?from={start_ts}&to={end_ts}&device={device}",
            headers={"x-api-key": API_KEY},
        ).json()
        data = data.json()
        rows += data["Records"]
    df = pd.DataFrame(rows, columns=data["Metadata"]["ColumnName"])
    df['device'] = device
    return df

In [ ]:
def get_device_measures(device_id, from_ts, to_ts, return_response=False):
    response = requests.get(
        url=f"https://jov3dcr05d.execute-api.ap-southeast-2.amazonaws.com/v1/sensordata?from={from_ts}&to={to_ts}&device={device_id}",
        headers=get_request_header(),
    )
    n_req = 1
    if return_response:
        return response
    data = response.json()
    rows = data["Records"]
    print(f"REQUEST: {n_req}")
    while data["ExecutionInfo"].get("NextToken"):
        token = data["ExecutionInfo"].get("NextToken")
        data = requests.get(
            url=f"https://jov3dcr05d.execute-api.ap-southeast-2.amazonaws.com/v1/sensordata?from={from_ts}&to={to_ts}&device={device_id}&nextToken={token}",
            headers=get_request_header(),
        ).json()
        n_req += 1
        print(f"REQUEST: {n_req}")
        rows += data["Records"]
    
    df = pd.DataFrame(rows, columns=data["Metadata"]["ColumnName"])
    df['device'] = device_id
    df = df.astype({"temperature": "float64", "co2": "float64", "iaq": "float64", "pressure": "float64", "humidity": "float64", "voc": "float64", "device": "str"})
    df.time = pd.to_datetime(df.time)
    df.time = df.time.dt.tz_localize('UTC').dt.tz_convert('Australia/ACT')
    return df.sort_values('time')

def get_device_predictions(device_id, from_ts, to_ts, return_response=False):
    response = requests.get(
        url=f"https://jov3dcr05d.execute-api.ap-southeast-2.amazonaws.com/v1/predictions?from={from_ts}&to={to_ts}&device={device_id}",
        headers=get_request_header(),
    )
    n_req = 1
    print(f"REQUEST: {n_req}")
    if return_response:
        return response
    data = response.json()
    rows = data["Records"]
    
    while data["ExecutionInfo"].get("NextToken"):
        token = data["ExecutionInfo"].get("NextToken")
        data = requests.get(
            url=f"https://jov3dcr05d.execute-api.ap-southeast-2.amazonaws.com/v1/predictions?from={from_ts}&to={to_ts}&device={device_id}&nextToken={token}",
            headers=get_request_header(),
        ).json()
        n_req += 1
        print(f"REQUEST: {n_req}")
        rows += data["Records"]
    
    df = pd.DataFrame(rows, columns=data["Metadata"]["ColumnName"])
    df['device'] = device_id
    df = df.astype({f"iaq~{i}": "float64" for i in range(1, 16)})
    df = df.astype({"device": "str"})
    df.time = pd.to_datetime(df.time)
    df.time = df.time.dt.tz_localize('UTC').dt.tz_convert('Australia/ACT')
    return df.sort_values('time')

In [ ]:
def get_device_measures_last(device_id, return_response=False, **kwargs):
    end = datetime.now().replace(second=0)
    start = end - timedelta(**kwargs)

    start_ts = int(datetime.timestamp(start))
    end_ts = int(datetime.timestamp(end))
    return get_device_measures(device_id, start_ts, end_ts, return_response=return_response)

def get_device_predictions_last(device_id, return_response=False, **kwargs):
    end = datetime.now().replace(second=0)
    start = end - timedelta(**kwargs)

    start_ts = int(datetime.timestamp(start))
    end_ts = int(datetime.timestamp(end))
    return get_device_predictions(device_id, start_ts, end_ts, return_response=return_response)

In [ ]:
def get_all_device_measures(from_ts, to_ts):
    devices = get_device_ids()
    df = pd.concat([get_device_measures(d, from_ts, to_ts) for d in devices])
    df = df.sort_values('time')
    return df

def get_all_device_predictions(from_ts, to_ts):
    devices = get_device_ids()
    df = pd.concat([get_device_predictions(d, from_ts, to_ts) for d in devices])
    df = df.sort_values('time')
    return df

In [ ]:
def get_all_device_measures_last(**kwargs):
    devices = get_device_ids()
    df = pd.concat([get_device_measures_last(d, **kwargs) for d in devices])
    df = df.sort_values('time')
    return df

def get_all_device_predictions_last(**kwargs):
    devices = get_device_ids()
    df = pd.concat([get_device_predictions_last(d, **kwargs) for d in devices])
    df = df.sort_values('time')
    return df